
# Ilusión de Mayoría en Redes Scale-Free — Fig. 2 ($r_{kk}$ estrictos)

**Objetivo:** reproducir la figura garantizando que la asortatividad $r_{kk}$ de cada red cumpla (±0.003) con los valores objetivo.



## Pseudocódigo
```
para cada α:
  para cada r_target:
    deg <- power-law
    G <- configuration_model -> grafo simple
    G <- rewire_to_rkk_strict(G, r_target, tol=0.003):
         - pasada guiada (steps_per_edge=8)
         - pulido (steps_per_edge=8)
         - si no llega: reintentos con semillas y mezcla neutra
    barrer ρ_kx:
         x <- asignar atributo con p=0.05 y ρ≈ρ_target (bisección en β)
         P>1/2 <- medir en GCC
graficar 3 paneles
```


In [1]:

import numpy as np
import networkx as nx
import scipy.sparse as sp
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional


In [2]:

@dataclass
class Config:
    N: int = 10_000
    alphas: Tuple[float, ...] = (2.1, 2.4, 3.1)
    r_by_alpha: Dict[float, Tuple[float, ...]] = None
    rho_grid: Tuple[float, ...] = tuple(np.round(np.linspace(0.0, 0.55, 12), 2))
    p_global: float = 0.05
    kmin: int = 2
    kmax: Optional[int] = None
    tol_r_strict: float = 0.003
    first_pass_steps_per_edge: float = 8.0
    polish_steps_per_edge: float = 8.0
    max_attempts: int = 4
    seed_base: int = 123
    tol_rho: float = 0.015
    max_bias_bisection: int = 24
    sample_nodes: Optional[int] = None
    domain_mode: str = "gcc"

CFG = Config()
CFG.r_by_alpha = {
    2.1: (-0.35, -0.25, -0.15, -0.05),
    2.4: (-0.20, -0.10,  0.00,  0.10,  0.20),
    3.1: (-0.15, -0.05,  0.00,  0.30),
}
CFG.kmax = int((CFG.N * 8)**0.5)
CFG


Config(N=10000, alphas=(2.1, 2.4, 3.1), r_by_alpha={2.1: (-0.35, -0.25, -0.15, -0.05), 2.4: (-0.2, -0.1, 0.0, 0.1, 0.2), 3.1: (-0.15, -0.05, 0.0, 0.3)}, rho_grid=(np.float64(0.0), np.float64(0.05), np.float64(0.1), np.float64(0.15), np.float64(0.2), np.float64(0.25), np.float64(0.3), np.float64(0.35), np.float64(0.4), np.float64(0.45), np.float64(0.5), np.float64(0.55)), p_global=0.05, kmin=2, kmax=282, tol_r_strict=0.003, first_pass_steps_per_edge=8.0, polish_steps_per_edge=8.0, max_attempts=4, seed_base=123, tol_rho=0.015, max_bias_bisection=24, sample_nodes=None, domain_mode='gcc')

In [3]:

def sample_powerlaw_degrees(N: int, alpha: float, kmin: int, kmax: int, rng):
    ks = np.arange(kmin, kmax + 1, dtype=int)
    pk = ks.astype(float) ** (-alpha); pk /= pk.sum()
    deg = rng.choice(ks, size=N, replace=True, p=pk)
    if deg.sum() % 2 == 1:
        i = rng.integers(0, N); deg[i] = min(deg[i] + 1, kmax)
    return deg

def config_graph_from_seq(deg: np.ndarray, seed: int = 42) -> nx.Graph:
    Gm = nx.configuration_model(deg, seed=seed)
    G = nx.Graph(Gm); G.remove_edges_from(nx.selfloop_edges(G))
    return G


In [4]:

def _assort_constants_from_degrees(G: nx.Graph, deg: Dict[int,int] = None):
    if deg is None: deg = dict(G.degree())
    M = G.number_of_edges()
    dvals = np.array(list(deg.values()), dtype=float)
    B = 0.5 * np.sum(dvals**2)
    C = 0.5 * np.sum(dvals**3)
    A = 0
    for u, v in G.edges():
        A += deg[u]*deg[v]
    denom = (C/M) - (B/M)**2
    return A, B, C, M, denom, deg

def _assort_from_A(A, B, C, M):
    return (A/M - (B/M)**2) / (C/M - (B/M)**2)

def _biased_edge_pick(edges, deg, rng, direction_sign):
    d = np.array([deg[u]+deg[v] for (u,v) in edges])
    q_hi = np.quantile(d, 0.8); q_lo = np.quantile(d, 0.2)
    hi_idx = np.where(d >= q_hi)[0]; lo_idx = np.where(d <= q_lo)[0]
    if direction_sign < 0:
        i = rng.choice(hi_idx) if hi_idx.size else rng.integers(len(edges))
        j = rng.choice(lo_idx) if lo_idx.size else rng.integers(len(edges))
    else:
        i = rng.integers(len(edges)); j = rng.integers(len(edges))
    return int(i), int(j)

def _rewire_pass(G: nx.Graph, r_target: float, steps_per_edge: float, tol: float, seed: int) -> float:
    rng = np.random.default_rng(seed)
    A, B, C, M, denom, deg = _assort_constants_from_degrees(G)
    if denom <= 0:
        return nx.degree_assortativity_coefficient(G)
    r = _assort_from_A(A, B, C, M)
    edges = list(G.edges())
    adj = {u: set(G.neighbors(u)) for u in G.nodes()}
    steps = int(steps_per_edge * M)
    direction = 1 if r_target > r else -1

    for _ in range(steps):
        i, j = _biased_edge_pick(edges, deg, rng, direction)
        if i == j: continue
        a,b = edges[i]; c,d = edges[j]
        if len({a,b,c,d}) < 4: continue
        valid1 = (c not in adj[a]) and (d not in adj[b]) and (a!=c) and (b!=d)
        valid2 = (d not in adj[a]) and (c not in adj[b]) and (a!=d) and (b!=c)
        if not (valid1 or valid2): continue

        ka,kb,kc,kd = deg[a],deg[b],deg[c],deg[d]
        current = ka*kb + kc*kd
        delta1 = (ka*kc + kb*kd) - current
        delta2 = (ka*kd + kb*kc) - current
        cand=[]; 
        if valid1: cand.append(("ac_bd", delta1))
        if valid2: cand.append(("ad_bc", delta2))
        if not cand: continue
        label, delta = max(cand, key=lambda x:x[1]) if direction>0 else min(cand, key=lambda x:x[1])
        if (direction>0 and delta<=0) or (direction<0 and delta>=0): 
            continue

        G.remove_edge(a,b); adj[a].remove(b); adj[b].remove(a)
        G.remove_edge(c,d); adj[c].remove(d); adj[d].remove(c)
        if label=="ac_bd":
            G.add_edge(a,c); adj[a].add(c); adj[c].add(a)
            G.add_edge(b,d); adj[b].add(d); adj[d].add(b)
            edges[i]=(a,c); edges[j]=(b,d)
        else:
            G.add_edge(a,d); adj[a].add(d); adj[d].add(a)
            G.add_edge(b,c); adj[b].add(c); adj[c].add(b)
            edges[i]=(a,d); edges[j]=(b,c)
        A += delta
        r = _assort_from_A(A,B,C,M)
        direction = 1 if r_target>r else -1
        if abs(r - r_target) <= tol:
            break
    return r

def rewire_to_rkk_strict(G: nx.Graph, r_target: float, cfg):
    for attempt in range(cfg.max_attempts):
        seed_pass = cfg.seed_base + 97*attempt
        r1 = _rewire_pass(G, r_target, cfg.first_pass_steps_per_edge, cfg.tol_r_strict, seed_pass)
        if abs(r1 - r_target) <= cfg.tol_r_strict:
            return r1
        r2 = _rewire_pass(G, r_target, cfg.polish_steps_per_edge, cfg.tol_r_strict, seed_pass+1)
        if abs(r2 - r_target) <= cfg.tol_r_strict:
            return r2
        # mezcla neutra para escapar de óptimos locales
        try:
            G = nx.double_edge_swap(G, nswap=int(0.5*G.number_of_edges()), max_tries=G.number_of_edges()*5)
        except Exception:
            pass
    return nx.degree_assortativity_coefficient(G)


In [5]:

def _pearson_binary_k(k, x):
    k = np.asarray(k, dtype=float)
    x = np.asarray(x, dtype=float)
    k_c = k - k.mean()
    x_c = x - x.mean()
    denom = np.sqrt(np.sum(k_c**2) * np.sum(x_c**2))
    if denom == 0:
        return 0.0
    return float(np.dot(k_c, x_c) / denom)

def assign_attribute_with_rho_and_p(k: np.ndarray, rho_target: float, p: float,
                                    tol: float, max_iter: int, rng):
    N = len(k); m = int(round(p * N))
    m = max(1, min(N-1, m))
    k = np.asarray(k, dtype=float)
    if np.any(k <= 0):
        pos = k[k > 0]
        kmin_pos = float(pos.min()) if pos.size > 0 else 1.0
        k_eff = k.copy(); k_eff[k_eff <= 0] = kmin_pos
    else:
        k_eff = k
    logk = np.log(k_eff)
    lo, hi = -12.0, +12.0
    x_best, rho_best = None, -1.0
    for _ in range(max_iter):
        beta = 0.5*(lo+hi)
        w = np.exp(beta * logk)
        w[~np.isfinite(w)] = 0.0
        s = w.sum()
        if not np.isfinite(s) or s <= 0.0:
            w = np.ones_like(w) / N
        else:
            w = w / s
        idx = np.random.default_rng().choice(N, size=m, replace=False, p=w)
        x = np.zeros(N, dtype=np.int8); x[idx]=1
        rho = _pearson_binary_k(k, x)
        x_best, rho_best = x, rho
        if abs(rho - rho_target) <= tol: break
        if rho < rho_target: lo = beta
        else: hi = beta
    return x_best, rho_best


In [6]:

def prob_majority(G: nx.Graph, x: np.ndarray, sample_nodes: Optional[int], domain: str, rng):
    if domain not in {"gcc", "all"}:
        raise ValueError("domain debe ser 'gcc' o 'all'")
    if domain == "gcc":
        GCC_nodes = max(nx.connected_components(G), key=len)
        H = G.subgraph(GCC_nodes).copy()
        nodes = np.array(list(H.nodes()), dtype=int)
        xH = x[nodes]
        A = nx.to_scipy_sparse_array(H, format="csr", dtype=np.int8)
        deg = np.asarray(A.sum(1)).ravel()
        ok = deg > 0
        if sample_nodes is not None and sample_nodes < ok.sum():
            idx = np.where(ok)[0]
            choose = rng.choice(idx, size=sample_nodes, replace=False)
            mask = np.zeros_like(ok, dtype=bool); mask[choose] = True
            ok = mask
        counts = (A @ xH)
        return (counts[ok] > 0.5 * deg[ok]).mean()
    else:
        A = nx.to_scipy_sparse_array(G, format="csr", dtype=np.int8)
        deg = np.asarray(A.sum(1)).ravel()
        ok = deg > 0
        if sample_nodes is not None and sample_nodes < ok.sum():
            idx = np.where(ok)[0]
            choose = rng.choice(idx, size=sample_nodes, replace=False)
            mask = np.zeros_like(ok, dtype=bool); mask[choose] = True
            ok = mask
        counts = (A @ x)
        return (counts[ok] > 0.5 * deg[ok]).mean()


In [7]:

def run_panel_strict(alpha: float, r_targets: Tuple[float, ...], cfg):
    results = {}; r_measured = {}
    rng = np.random.default_rng(cfg.seed_base)
    for idx, r_target in enumerate(r_targets):
        deg = sample_powerlaw_degrees(cfg.N, alpha, cfg.kmin, cfg.kmax, rng)
        G = config_graph_from_seq(deg, seed=cfg.seed_base + 31*idx)
        r_real = rewire_to_rkk_strict(G, r_target, cfg)
        r_measured[r_target] = r_real
        if abs(r_real - r_target) > cfg.tol_r_strict:
            print(f"[ADVERTENCIA] α={alpha}, r_target={r_target:+.2f}, r_medido={r_real:+.4f}")
        k = np.array([d for _, d in sorted(G.degree(), key=lambda x: x[0])], dtype=int)
        rho_meas, pmaj_vals = [], []
        for rho_target in cfg.rho_grid:
            x, rho_real = assign_attribute_with_rho_and_p(k, rho_target, cfg.p_global, cfg.tol_rho, cfg.max_bias_bisection, rng)
            p = prob_majority(G, x, cfg.sample_nodes, cfg.domain_mode, rng)
            rho_meas.append(float(rho_real)); pmaj_vals.append(float(p))
        order = np.argsort(rho_meas)
        results[r_target] = (np.array(rho_meas)[order], np.array(pmaj_vals)[order])
    return results, r_measured

def run_all_panels_strict(cfg):
    results_by_alpha, r_meas_by_alpha = {}, {}
    for a in cfg.alphas:
        res, r_meas = run_panel_strict(a, cfg.r_by_alpha[a], cfg)
        results_by_alpha[a] = res; r_meas_by_alpha[a] = r_meas
    return results_by_alpha, r_meas_by_alpha

def plot_three_panels(results_by_alpha, r_meas_by_alpha, cfg, use_target_labels=False):
    fig, axes = plt.subplots(1, 3, figsize=(13.5, 4), dpi=120, sharex=True, sharey=True)
    marker_sets = {2.1:["o","^","s","x"], 2.4:["o","^","s","D","x"], 3.1:["o","^","s","x"]}
    titles = {2.1:r"(a) $\alpha=2.1$", 2.4:r"(b) $\alpha=2.4$", 3.1:r"(c) $\alpha=3.1$"}
    for ax, a in zip(axes, cfg.alphas):
        markers = marker_sets[a]
        for i, r_target in enumerate(cfg.r_by_alpha[a]):
            rhos, vals = results_by_alpha[a][r_target]
            lab = rf"$r_{{kk}}={r_target:+.2f}$" if use_target_labels else rf"$r_{{kk}}={r_meas_by_alpha[a][r_target]:+.2f}$"
            ax.plot(rhos, vals, marker=markers[i%len(markers)], linewidth=1.8, markersize=5.5, label=lab)
        ax.set_title(titles[a]); ax.grid(True, alpha=.25, linestyle="--")
        ax.set_xlim(min(cfg.rho_grid)-0.02, max(cfg.rho_grid)+0.02); ax.set_ylim(0,1.0)
        ax.set_xlabel(r"k–x correlation, $\rho_{kx}$")
        if a == cfg.alphas[0]:
            ax.set_ylabel(r"Probability of majority, $P_{>1/2}$")
        ax.legend(frameon=True, loc="upper left", fontsize=9)
    plt.tight_layout(); plt.show()


In [ ]:

# Ejecutar todo (puedes activar muestreo para acelerar)
# CFG.sample_nodes = 4000
results_by_alpha, r_meas_by_alpha = run_all_panels_strict(CFG)
plot_three_panels(results_by_alpha, r_meas_by_alpha, CFG, use_target_labels=False)
